In [8]:
from __future__ import annotations

import argparse
import csv
import json
import random
from datetime import datetime
from pathlib import Path
from typing import List, Optional, Sequence, Tuple

import matplotlib

matplotlib.use("Agg")
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor

In [9]:
def parse_args(argv: Sequence[str] | None = None) -> argparse.Namespace:
    parser = argparse.ArgumentParser(
        description="Train an XGBoost regressor for cell counting."
    )
    parser.add_argument(
        "--metadata",
        type=Path,
        default=Path("processed/metadata.csv"),
        help="CSV generated by data_preprocess.py.",
    )
    parser.add_argument(
        "--root",
        type=Path,
        default=Path("processed"),
        help="Root directory containing sample NPZ files.",
    )
    parser.add_argument(
        "--experiment-root",
        type=Path,
        default=Path("experiments"),
        help="Directory where run artifacts will be stored.",
    )
    parser.add_argument(
        "--run-name",
        type=str,
        default=None,
        help="Optional identifier for this training run (defaults to timestamp).",
    )
    parser.add_argument(
        "--val-split",
        type=float,
        default=0.1,
        help="Fraction reserved for validation.",
    )
    parser.add_argument(
        "--test-split",
        type=float,
        default=0.1,
        help="Fraction reserved for final testing.",
    )
    parser.add_argument(
        "--epochs",
        type=int,
        default=200,
        help="Number of boosting rounds (mirrors cnn.py epochs).",
    )
    parser.add_argument(
        "--learning-rate",
        type=float,
        default=0.1,
        help="XGBoost learning rate (eta).",
    )
    parser.add_argument(
        "--max-depth",
        type=int,
        default=6,
        help="Maximum tree depth.",
    )
    parser.add_argument(
        "--subsample",
        type=float,
        default=0.8,
        help="Row subsample ratio per tree.",
    )
    parser.add_argument(
        "--colsample-bytree",
        type=float,
        default=0.8,
        help="Column subsample ratio per tree.",
    )
    parser.add_argument(
        "--reg-alpha",
        type=float,
        default=0.0,
        help="L1 regularization term.",
    )
    parser.add_argument(
        "--reg-lambda",
        type=float,
        default=1.0,
        help="L2 regularization term.",
    )
    parser.add_argument(
        "--min-child-weight",
        type=float,
        default=1.0,
        help="Minimum sum of instance weight needed in a child.",
    )
    parser.add_argument(
        "--tree-method",
        type=str,
        default="hist",
        help="Tree construction algorithm.",
    )
    parser.add_argument(
        "--seed",
        type=int,
        default=42,
        help="Random seed.",
    )
    return parser.parse_args(args=[] if argv is None else list(argv))

In [10]:
def load_metadata(metadata_path: Path, root: Path) -> List[Path]:
    npz_paths: List[Path] = []
    with metadata_path.open("r", newline="", encoding="utf-8") as fh:
        reader = csv.DictReader(fh)
        for row in reader:
            sample_rel = Path(row["sample_npz"])
            if sample_rel.is_absolute():
                sample_path = sample_rel
            elif sample_rel.parts and sample_rel.parts[0] == root.name:
                sample_path = (root.parent / sample_rel).resolve()
            else:
                sample_path = (root / sample_rel).resolve()
            npz_paths.append(sample_path)
    if not npz_paths:
        raise SystemExit(f"No samples found via {metadata_path}.")
    return npz_paths

In [11]:
def load_dataset(npz_paths: Sequence[Path]) -> Tuple[np.ndarray, np.ndarray]:
    features: List[np.ndarray] = []
    targets: List[float] = []
    for path in npz_paths:
        with np.load(path) as data:
            image = data["image"]
            features.append(image.reshape(-1))
            targets.append(float(np.array(data["count"]).item()))
    # store as float16 to cut memory footprint before training
    X = np.stack(features).astype(np.float16)
    y = np.asarray(targets, dtype=np.float32)
    return X, y


def create_run_dir(root: Path, run_name: str | None) -> Path:
    root = root.resolve()
    root.mkdir(parents=True, exist_ok=True)
    if run_name:
        base = run_name.strip().replace(" ", "_") or datetime.now().strftime("%Y%m%d_%H%M%S")
    else:
        base = datetime.now().strftime("%Y%m%d_%H%M%S")
    run_dir = root / base
    suffix = 1
    while run_dir.exists():
        run_dir = root / f"{base}_{suffix:02d}"
        suffix += 1
    run_dir.mkdir()
    return run_dir


def serialize_config(
    args: argparse.Namespace,
    train_count: int,
    val_count: int,
    test_count: int,
) -> dict:
    return {
        "metadata": str(args.metadata),
        "root": str(args.root),
        "epochs": args.epochs,
        "learning_rate": args.learning_rate,
        "max_depth": args.max_depth,
        "subsample": args.subsample,
        "colsample_bytree": args.colsample_bytree,
        "reg_alpha": args.reg_alpha,
        "reg_lambda": args.reg_lambda,
        "min_child_weight": args.min_child_weight,
        "tree_method": args.tree_method,
        "val_split": args.val_split,
        "test_split": args.test_split,
        "seed": args.seed,
        "train_samples": train_count,
        "val_samples": val_count,
        "test_samples": test_count,
    }

In [12]:
def main(argv: Optional[Sequence[str]] = None) -> None:
    args = parse_args(argv)

    if args.epochs <= 0:
        raise SystemExit("epochs must be positive.")

    random.seed(args.seed)
    np.random.seed(args.seed)

    npz_paths = load_metadata(args.metadata, args.root)
    X, y = load_dataset(npz_paths)

    if args.test_split > 0.0:
        X_temp, X_test, y_temp, y_test = train_test_split(
            X,
            y,
            test_size=args.test_split,
            random_state=args.seed,
        )
    else:
        X_temp, y_temp = X, y
        X_test, y_test = None, None

    if args.val_split > 0.0:
        denom = 1.0 - args.test_split
        denom = denom if denom > 0 else 1.0
        val_fraction = args.val_split / denom
        X_train, X_val, y_train, y_val = train_test_split(
            X_temp,
            y_temp,
            test_size=val_fraction,
            random_state=args.seed,
        )
    else:
        X_train, y_train = X_temp, y_temp
        X_val, y_val = None, None

    eval_set: List[Tuple[np.ndarray, np.ndarray]] = [(X_train, y_train)]
    if X_val is not None:
        eval_set.append((X_val, y_val))

    model = XGBRegressor(
        n_estimators=args.epochs,
        learning_rate=args.learning_rate,
        max_depth=args.max_depth,
        subsample=args.subsample,
        colsample_bytree=args.colsample_bytree,
        reg_alpha=args.reg_alpha,
        reg_lambda=args.reg_lambda,
        min_child_weight=args.min_child_weight,
        random_state=args.seed,
        tree_method=args.tree_method,
        objective="reg:squarederror",
        eval_metric=["rmse", "mae"],
    )

    model.fit(
        X_train,
        y_train,
        eval_set=eval_set,
        verbose=True,
    )

    evals_result = model.evals_result()
    train_rmse_history = [float(v) for v in evals_result["validation_0"]["rmse"]]
    num_epochs = len(train_rmse_history)
    if num_epochs == 0:
        raise SystemExit("No training history recorded by XGBoost.")

    if X_val is not None:
        val_rmse_history = [float(v) for v in evals_result["validation_1"]["rmse"]]
        val_mae_history = [float(v) for v in evals_result["validation_1"]["mae"]]
    else:
        val_rmse_history = [float("nan")] * num_epochs
        val_mae_history = [float("nan")] * num_epochs

    run_dir = create_run_dir(args.experiment_root, args.run_name)
    history: List[dict] = []

    config = serialize_config(
        args,
        train_count=len(X_train),
        val_count=len(X_val) if X_val is not None else 0,
        test_count=len(X_test) if X_test is not None else 0,
    )
    with (run_dir / "config.json").open("w", encoding="utf-8") as cfg_file:
        json.dump(config, cfg_file, indent=2)

    for epoch_idx in range(num_epochs):
        epoch = epoch_idx + 1
        progress = (epoch_idx + 1) / num_epochs
        train_mse = float(train_rmse_history[epoch_idx]) ** 2
        val_rmse = float(val_rmse_history[epoch_idx])
        val_mae = float(val_mae_history[epoch_idx])
        val_mse = val_rmse ** 2 if not np.isnan(val_rmse) else float("nan")

        print(
            f"Epoch {epoch:03d} | "
            f"{progress * 100:6.2f}% of epoch complete "
            f"(step {epoch}/{num_epochs})"
        )
        if np.isnan(val_mse) or np.isnan(val_mae):
            msg = f"Epoch {epoch:03d} - train MSE: {train_mse:.4f}, val MSE: nan, val MAE: nan"
        else:
            msg = (
                f"Epoch {epoch:03d} - train MSE: {train_mse:.4f}, "
                f"val MSE: {val_mse:.4f}, val MAE: {val_mae:.4f}"
            )
        print(f"{progress * 100:6.2f}% | {msg}")
        history.append(
            {
                "epoch": epoch,
                "train_mse": float(train_mse),
                "val_mse": float(val_mse),
                "val_mae": float(val_mae),
            }
        )

    metrics_path = run_dir / "metrics.csv"
    with metrics_path.open("w", newline="", encoding="utf-8") as metrics_file:
        writer = csv.DictWriter(
            metrics_file,
            fieldnames=["epoch", "train_mse", "val_mse", "val_mae"],
        )
        writer.writeheader()
        writer.writerows(history)

    scatter_path: Optional[Path] = None
    test_metrics_path: Optional[Path] = None
    if X_test is not None and X_test.size > 0:
        test_preds = model.predict(X_test)
        test_mse = float(mean_squared_error(y_test, test_preds))
        test_mae = float(mean_absolute_error(y_test, test_preds))
        print(f"Test MSE: {test_mse:.4f} | Test MAE: {test_mae:.4f}")
        test_metrics_path = run_dir / "test_metrics.json"
        with test_metrics_path.open("w", encoding="utf-8") as fh:
            json.dump({"mse": test_mse, "mae": test_mae}, fh, indent=2)

        scatter_path = run_dir / "test_scatter.png"
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.scatter(y_test, test_preds, s=12, alpha=0.6, edgecolors="none")
        min_val = float(min(test_preds.min(), y_test.min()))
        max_val = float(max(test_preds.max(), y_test.max()))
        ax.plot([min_val, max_val], [min_val, max_val], color="gray", linestyle="--")
        ax.set_xlabel("True Count")
        ax.set_ylabel("Predicted Count")
        ax.set_title("Test Predictions vs. Ground Truth")
        ax.grid(True, alpha=0.2)
        fig.tight_layout()
        fig.savefig(scatter_path, dpi=200)
        plt.close(fig)

    model_path = run_dir / "model.pt"
    model.save_model(str(model_path))
    print(f"Saved model weights to {model_path}")
    print(f"Metrics logged to {metrics_path}")
    if test_metrics_path is not None:
        print(f"Test metrics saved to {test_metrics_path}")
    if scatter_path is not None:
        print(f"Test scatter plot saved to {scatter_path}")
    print(f"Run artifacts stored in {run_dir}")

In [ ]:
# Launch training with lighter defaults to reduce memory load.
main(["--epochs", "80", "--max-depth", "4", "--subsample", "0.5", "--colsample-bytree", "0.5"])
